In [16]:
import pandas as pd
import numpy as np
np.random.seed(256)

df = pd.read_csv('./loan.csv')
df['loanID'] = np.random.randint(100000000, size=len(df))
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,loanID
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,62422176
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,94046906
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,67602160
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,93385845
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,8416620


### Reduce columns
There are a lot of columns that are post-loan and would be target leakers.<br>

In [17]:
cols = df.columns[0:36].insert(0,'loanID')
print(cols)

Index(['loanID', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url',
       'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status'],
      dtype='object')


### Format the target
Loan_status is default, paid, & current. <br>
We want to change anything not "default" to 0.

In [19]:
df['loan_status'].value_counts()

Fully Paid                                             1041952
Current                                                 919695
Charged Off                                             261655
Late (31-120 days)                                       21897
In Grace Period                                           8952
Late (16-30 days)                                         3737
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     31
Name: loan_status, dtype: int64

In [20]:
new = df[cols]
new['is_bad'] = [1 if i=='Default' or i=='Charged Off' else 0 for i in new['loan_status']]
new['is_bad'].value_counts()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0    1998982
1     261686
Name: is_bad, dtype: int64

### Extract year so we can focus on mature loans...
Newer loans haven't had a chance to default yet. <br>
For this reason, we want only loans that are pre-2017.

In [21]:
new['year']=new['issue_d'].str.replace('^.*-','')
new['year'].value_counts()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


2018    495242
2017    443579
2016    434407
2015    421095
2014    235629
2013    134814
2012     53367
2011     21721
2010     12537
2009      5281
2008      2393
2007       603
Name: year, dtype: int64

### Drop columns and write to file
Remove id columns and loan status and date field<br>
File #1: a 5% sample for training<br>
File #2: a 1% sample used for scoring along with associationId<br>
Write to csv's using date filter

In [22]:
final_drops=['loan_status','id','member_id','issue_d','year']
final = new[new['year'] < '2016'].drop(final_drops,axis=1)
final.sample(frac=0.05, replace=False, random_state=99 ).to_csv('training.csv',index=False)
final.sample(frac=0.01, replace=False, random_state=11 ).to_csv('scoring.csv',index=False)
